In [7]:
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Activation, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
import scipy
import math
import sys
sys.path.append("..")

In [39]:
# method 1
from model import get_training_model_eggnog
model = get_training_model_eggnog(weight_decay, gpus=use_multiple_gpus)

NameError: name 'weight_decay' is not defined

In [34]:
# method 2

def relu(x): 
    return Activation('relu')(x)

def conv(x, nf, ks, name):
    x1 = Conv2D(nf, (ks, ks), padding='same', name=name)(x)
    return x1

def pooling(x, ks, st, name):
    x = MaxPooling2D((ks, ks), strides=(st, st), name=name)(x)
    return x

def vgg_block(x):
     
    # Block 1
    x = conv(x, 64, 3, "conv1_1")
    x = relu(x)
    x = conv(x, 64, 3, "conv1_2")
    x = relu(x)
    x = pooling(x, 2, 2, "pool1_1")

    # Block 2
    x = conv(x, 128, 3, "conv2_1")
    x = relu(x)
    x = conv(x, 128, 3, "conv2_2")
    x = relu(x)
    x = pooling(x, 2, 2, "pool2_1")
    
    # Block 3
    x = conv(x, 256, 3, "conv3_1")
    x = relu(x)    
    x = conv(x, 256, 3, "conv3_2")
    x = relu(x)    
    x = conv(x, 256, 3, "conv3_3")
    x = relu(x)    
    x = conv(x, 256, 3, "conv3_4")
    x = relu(x)    
    x = pooling(x, 2, 2, "pool3_1")
    
    # Block 4
    x = conv(x, 512, 3, "conv4_1")
    x = relu(x)    
    x = conv(x, 512, 3, "conv4_2")
    x = relu(x)    
    
    # Additional non vgg layers
    x = conv(x, 256, 3, "conv4_3_CPM")
    x = relu(x)
    x = conv(x, 128, 3, "conv4_4_CPM")
    x = relu(x)
    
    return x

def stage1_block(x, num_p, branch):
    
    # Block 1        
    x = conv(x, 128, 3, "conv5_1_CPM_L%d" % branch)
    x = relu(x)
    x = conv(x, 128, 3, "conv5_2_CPM_L%d" % branch)
    x = relu(x)
    x = conv(x, 128, 3, "conv5_3_CPM_L%d" % branch)
    x = relu(x)
    x = conv(x, 512, 1, "conv5_4_CPM_L%d" % branch)
    x = relu(x)
    x = conv(x, num_p, 1, "conv5_5_CPM_L%d" % branch)
    
    return x

def stageT_block(x, num_p, stage, branch):
        
    # Block 1        
    x = conv(x, 128, 7, "Mconv1_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 7, "Mconv2_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 7, "Mconv3_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 7, "Mconv4_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 7, "Mconv5_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 1, "Mconv6_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, num_p, 1, "Mconv7_stage%d_L%d" % (stage, branch))
    
    return x

In [37]:
# weights_path = "../model/keras/model.h5"  # orginal weights converted from caffe
weights_path = "../model/keras/weights.0021.h5"
# weights_path = "training/weights.best.h5" # weights tarined from scratch 

input_shape = (None,None,3)

img_input = Input(shape=input_shape)

stages = 6
np_branch1 = 38
np_branch2 = 19

img_normalized = Lambda(lambda x: x / 256 - 0.5)(img_input)  # [-0.5, 0.5]

# VGG
stage0_out = vgg_block(img_normalized)

# stage 1
stage1_branch1_out = stage1_block(stage0_out, np_branch1, 1)
stage1_branch2_out = stage1_block(stage0_out, np_branch2, 2)
x = Concatenate()([stage1_branch1_out, stage1_branch2_out, stage0_out])

# stage t >= 2
for sn in range(2, stages + 1):
    stageT_branch1_out = stageT_block(x, np_branch1, sn, 1)
    stageT_branch2_out = stageT_block(x, np_branch2, sn, 2)
    if (sn < stages):
        x = Concatenate()([stageT_branch1_out, stageT_branch2_out, stage0_out])

model = Model(img_input, [stageT_branch1_out, stageT_branch2_out])
model.load_weights(weights_path, by_name=True)



In [38]:
print("loaded ", weights_path)
for layer in model.layers:
        if "conv" in layer.name:
            print("================================\n", layer.name)
            print("layer weights shape", layer.get_weights()[0].shape)
            print("layer bias shape", layer.get_weights()[1].shape)

loaded  ../model/keras/weights.0021.h5
 conv1_1
layer weights shape (3, 3, 3, 64)
layer bias shape (64,)
 conv1_2
layer weights shape (3, 3, 64, 64)
layer bias shape (64,)
 conv2_1
layer weights shape (3, 3, 64, 128)
layer bias shape (128,)
 conv2_2
layer weights shape (3, 3, 128, 128)
layer bias shape (128,)
 conv3_1
layer weights shape (3, 3, 128, 256)
layer bias shape (256,)
 conv3_2
layer weights shape (3, 3, 256, 256)
layer bias shape (256,)
 conv3_3
layer weights shape (3, 3, 256, 256)
layer bias shape (256,)
 conv3_4
layer weights shape (3, 3, 256, 256)
layer bias shape (256,)
 conv4_1
layer weights shape (3, 3, 256, 512)
layer bias shape (512,)
 conv4_2
layer weights shape (3, 3, 512, 512)
layer bias shape (512,)
 conv4_3_CPM
layer weights shape (3, 3, 512, 256)
layer bias shape (256,)
 conv4_4_CPM
layer weights shape (3, 3, 256, 128)
layer bias shape (128,)
 Mconv1_stage1_L1
layer weights shape (3, 3, 128, 128)
layer bias shape (128,)
 Mconv1_stage1_L2
layer weights shape (3, 

layer weights shape (1, 1, 128, 128)
layer bias shape (128,)
 Mconv6_stage5_L2
layer weights shape (1, 1, 128, 128)
layer bias shape (128,)
 Mconv7_stage5_L1
layer weights shape (1, 1, 128, 38)
layer bias shape (38,)
 Mconv7_stage5_L2
layer weights shape (1, 1, 128, 19)
layer bias shape (19,)
 Mconv1_stage6_L1
layer weights shape (7, 7, 185, 128)
layer bias shape (128,)
 Mconv1_stage6_L2
layer weights shape (7, 7, 185, 128)
layer bias shape (128,)
 Mconv2_stage6_L1
layer weights shape (7, 7, 128, 128)
layer bias shape (128,)
 Mconv2_stage6_L2
layer weights shape (7, 7, 128, 128)
layer bias shape (128,)
 Mconv3_stage6_L1
layer weights shape (7, 7, 128, 128)
layer bias shape (128,)
 Mconv3_stage6_L2
layer weights shape (7, 7, 128, 128)
layer bias shape (128,)
 Mconv4_stage6_L1
layer weights shape (7, 7, 128, 128)
layer bias shape (128,)
 Mconv4_stage6_L2
layer weights shape (7, 7, 128, 128)
layer bias shape (128,)
 Mconv5_stage6_L1
layer weights shape (7, 7, 128, 128)
layer bias shape (1